In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker, relationship, join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from statsmodels.regression.quantile_regression import QuantReg
import mpld3
from PIL import Image
from copy import copy

from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D
import pickle
from PIL import Image, ImageDraw

import sys
sys.path.append('/root/alok/repos/cv_research/alok/biomass_estimation/production_data_analysis_v3')
from template_matching import enhance, find_matches_and_homography, adjust_keypoints


pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 50)

<h1> Download Data </h1>

In [ ]:
# AWS credentials
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")
s3_access_utils = S3AccessUtils('/root/data')


# prod SQL credentaials
sql_credentials = json.load(open(os.environ["PROD_RESEARCH_SQL_CREDENTIALS"]))
rds_access_utils = RDSAccessUtils(sql_credentials)

sql_query = '''
select * from keypoint_annotations
where pen_id = 7
and keypoints is not NULL;
'''

original_df = rds_access_utils.extract_from_database(sql_query)

In [ ]:
def get_world_keypoints(row):
    if 'leftCrop' in row.keypoints and 'rightCrop' in row.keypoints:
        return pixel2world(row.keypoints['leftCrop'], row.keypoints['rightCrop'], row.camera_metadata)
    else:
        return None
    
original_df['world_keypoints'] = original_df.apply(
    lambda x: get_world_keypoints(x), axis=1
)

In [ ]:
f = '/root/data/temp/results_557ec1732d8bc8bc66951d2ea4e69b935d69b111_model_lateral_only_original_bremnes_data.h5'
original_df = pd.read_hdf(f, 'table') 

In [ ]:
original_df.sort_values('estimated_biomass_g', ascending=False)

In [ ]:
original_df.iloc[5618].estimated_biomass_g

In [ ]:
idx = 0
left_image_f, _, _ = s3_access_utils.download_from_url(original_df.left_image_url.iloc[idx])
right_image_f, _, _ = s3_access_utils.download_from_url(original_df.right_image_url.iloc[idx])
keypoints = original_df.keypoints.iloc[idx]
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)


In [ ]:
good, matchesMask, H = find_matches_and_homography(imageL, imageR)
# adjusted_keypoints = adjust_keypoints(keypoints, H)

In [ ]:
adjusted_keypoints

In [ ]:
imageL.shape

In [ ]:
imageL.shape

In [ ]:
imageL.shape

<h1> SIFT Based Enhancement Techiques </h1>

In [ ]:
# SIFT based correction - functions

def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image 




def draw_matches(img1, kp1, img2, kp2, matches, matchesMask, color=None, drawFeatures=True): 
    if len(img1.shape) == 3:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], img1.shape[2])
    elif len(img1.shape) == 2:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1])
    new_img = np.zeros(new_shape, type(img1.flat[0]))  
    # Place images onto the new image.
    new_img[0:img1.shape[0],0:img1.shape[1]] = img1
    new_img[0:img2.shape[0],img1.shape[1]:img1.shape[1]+img2.shape[1]] = img2
    
    if drawFeatures==False:
        return new_img

    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    r = 15
    thickness = 3
    if color:
        c = color
    i=0
    for m in matches:
        i=i+1
        # Generate random color for RGB/BGR and grayscale images as needed.
        if not color: 
            c = np.random.randint(0,256,3) if len(img1.shape) == 3 else np.random.randint(0,256)
            c = tuple([int(x) for x in c])        
        if matchesMask[i-1]==0: 
            continue
        end1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
        end2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int) + np.array([img1.shape[1], 0]))
        cv2.line(new_img, end1, end2, c, thickness, )
        cv2.circle(new_img, end1, r, c, thickness)
        cv2.circle(new_img, end2, r, c, thickness)
    return new_img





<h1> Try SIFT base matching </h1

In [ ]:
def _generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

def _normalize_world_keypoints(wkps, rotate=True):
    body_parts = wkps.keys()
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['HYPURAL_PLATE'] for bp in body_parts}

    if not rotate:
        return translated_wkps
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp].T) for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
    v = norm_wkps_intermediate['ADIPOSE_FIN'] - np.array([norm_wkps_intermediate['ADIPOSE_FIN'][0], 0, 0])
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp]) for bp in body_parts}
    
    # perform reflecton if necessary
    if norm_wkps['PECTORAL_FIN'][1] > 0:
        norm_wkps = {bp: np.array([
            norm_wkps[bp][0],
            -norm_wkps[bp][1],
            norm_wkps[bp][2]
        ]) for bp in body_parts}
    
    return norm_wkps


def draw_matches_3D(img1, kp1, img2, kp2, matches, matchesMask): 
    

    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    i=0
    wkps = []
    for m in matches:
        # Generate random color for RGB/BGR and grayscale images as needed.
        
        if matchesMask[i] == 1:
            p1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
            p2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int))
            p1_x_frame = p1[0] + original_df.left_crop_metadata.iloc[idx]['x_coord']
            p1_y_frame = p1[1] + original_df.left_crop_metadata.iloc[idx]['y_coord']
            p2_x_frame = p2[0] + original_df.right_crop_metadata.iloc[idx]['x_coord']
            params = original_df.camera_metadata.iloc[idx]
            disp = abs(p1_x_frame - p2_x_frame)
            depth = depth_from_disp(disp, params)
            wkp = convert_to_world_point(p1_y_frame, p1_x_frame, depth, params)
            wkps.append(wkp)
        i += 1
        
    return wkps


In [ ]:
idx = 5154
left_image_f, _, _ = s3_access_utils.download_from_url(original_df.left_image_url.iloc[idx])
right_image_f, _, _ = s3_access_utils.download_from_url(original_df.right_image_url.iloc[idx])
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)


In [ ]:
MIN_MATCH_COUNT = 11
GOOD_PERC = 0.7

sift = cv2.KAZE_create()
img1 = enhance(imageL)
img2 = enhance(imageR)
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
for m,n in matches:
    if m.distance < GOOD_PERC*n.distance:
        good.append(m)
if len(good)>=MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None

In [ ]:
%matplotlib inline
img3 = draw_matches(img1,kp1,img2,kp2,good,matchesMask,None,False)
img3o = draw_matches(img1,kp1,img2,kp2,good,matchesMask,None,True)
alpha = 0.7  # Transparency factor.
img3 = cv2.addWeighted(img3o, alpha, img3, 1 - alpha, 0)

f, ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(img3)
ax.axis("off")
plt.show()

In [ ]:
body_wkps = draw_matches_3D(img1, kp1, img2, kp2, good, matchesMask)

In [ ]:
sum(matchesMask)/len(matchesMask)

In [ ]:
wkps = original_df.world_keypoints.iloc[idx]
wkps['BODY'] = np.array(body_wkps)


In [ ]:
norm_wkps = _normalize_world_keypoints(wkps)

In [ ]:
%matplotlib notebook
body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
xs = [norm_wkps[bp][0] for bp in body_parts]
ys = [norm_wkps[bp][1] for bp in body_parts]
zs = [norm_wkps[bp][2] for bp in body_parts]
xs.extend(list(norm_wkps['BODY'][0]))
ys.extend(list(norm_wkps['BODY'][1]))
zs.extend(list(norm_wkps['BODY'][2]))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, max(xs))
ax.set_ylim3d(-0.3, 0.3)
ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

In [ ]:
keypoints = original_df.keypoints.iloc[idx]
left_keypoints_crop = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['leftCrop']}
right_keypoints_crop = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['rightCrop']}

# adjust left and right keypoints
left_keypoints_crop_adjusted, right_keypoints_crop_adjusted = [], []
for i, bp in enumerate([item['keypointType'] for item in keypoints['leftCrop']]):
    kpL = left_keypoints_crop[bp]
    ptx = np.array([kpL[0], kpL[1], 1])
    zx = np.dot(M, ptx)
    kpL2R = [zx[0] / zx[2], zx[1] / zx[2]]

    kpR = right_keypoints_crop[bp]
    pty = np.array([kpR[0], kpR[1], 1])
    zy = np.dot(np.linalg.inv(M), pty)
    kpR2L = [zy[0] / zy[2], zy[1] / zy[2]]

#     kpL_adjusted = [(kpL[0] + kpL2R[0]) / 2.0, (kpL[1] + kpL2R[1]) / 2.0]
#     kpR_adjusted = [(kpR[0] + kpR2L[0]) / 2.0, (kpR[1] + kpR2L[1]) / 2.0]
    kpL_adjusted = [(kpL[0] + kpR2L[0]) / 2.0, (kpL[1] + kpR2L[1]) / 2.0]
    kpR_adjusted = [(kpR[0] + kpL2R[0]) / 2.0, (kpR[1] + kpL2R[1]) / 2.0]
    item_left = keypoints['leftCrop'][i]
    item_right = keypoints['rightCrop'][i]

    new_item_left = {
        'keypointType': bp,
        'xCrop': kpL_adjusted[0],
        'xFrame': item_left['xFrame'] - item_left['xCrop'] + kpL_adjusted[0],
        'yCrop': kpL_adjusted[1],
        'yFrame': item_left['yFrame'] - item_left['yCrop'] + kpL_adjusted[1]
    }
    left_keypoints_crop_adjusted.append(new_item_left)

    new_item_right = {
        'keypointType': bp,
        'xCrop': kpR_adjusted[0],
        'xFrame': item_right['xFrame'] - item_right['xCrop'] + kpR_adjusted[0],
        'yCrop': kpR_adjusted[1],
        'yFrame': item_right['yFrame'] - item_right['yCrop'] + kpR_adjusted[1]
    }
    right_keypoints_crop_adjusted.append(new_item_right)



    

In [ ]:
wkps = pixel2world(left_keypoints_crop_adjusted, right_keypoints_crop_adjusted, original_df.camera_metadata.iloc[idx])
norm_wkps = _normalize_world_keypoints(wkps)

In [ ]:
%matplotlib notebook
body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
xs = [norm_wkps[bp][0] for bp in body_parts]
ys = [norm_wkps[bp][1] for bp in body_parts]
zs = [norm_wkps[bp][2] for bp in body_parts]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, max(xs))
ax.set_ylim3d(-0.3, 0.3)
ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

In [ ]:
man_dists=[]
kpL = []
kpR = []    
kpL2R = []
kpR2L = [] 
im1ps = []
im2ps = []
gtL2R = []
gtR2L = [] 
gt1ps = []
gt2ps = []
Hx=M
Hy=np.linalg.inv(M)
for c in range(FLAGS.joints):
    hm = cv2.resize(final_stage_heatmapL[..., c], (widthL, heightL))
    hm_maxL = list(np.where(hm == hm.max()))   
    kpL.append([int(hm_maxL[1][0]), int(hm_maxL[0][0])]) 
    ptx=np.array([kpL[c][0],kpL[c][1],1])
    zx=np.dot(Hx,ptx)
    kpL2R.append([int(zx[0]/zx[2]), int(zx[1]/zx[2])]) 
    gtx=np.array([gtkeypointsL[c][0],gtkeypointsL[c][1],1])
    gzx=np.dot(Hx,gtx)
    gtL2R.append([int(gzx[0]/gzx[2]), int(gzx[1]/gzx[2])]) 
#         gt_kp = gtkeypointsL[c,:]
#         man_distL = np.sqrt(pow(hm_maxL[1][0] - gt_kp[0],2)+pow(hm_maxL[0][0] - gt_kp[1],2))
#         man_dists.append(man_distL)
#         print("Left crop Manhattan distance between pred and gt {} for {}".format(man_distL[0], FLAGS.keypoints_order[c]))
    hm = cv2.resize(final_stage_heatmapR[..., c], (widthR, heightR))
    hm_maxR = np.where(hm == hm.max())
    kpR.append([int(hm_maxR[1][0]), int(hm_maxR[0][0])])
    pty=np.array([kpR[c][0],kpR[c][1],1])
    zy=np.dot(Hy,pty)
    kpR2L.append([int(zy[0]/zy[2]), int(zy[1]/zy[2])]) 
    gty=np.array([gtkeypointsR[c][0],gtkeypointsR[c][1],1])
    gzy=np.dot(Hy,gty)
    gtR2L.append([int(gzy[0]/gzy[2]), int(gzy[1]/gzy[2])])
#         gt_kp = gtkeypointsR[c,:]
#         man_distR = np.sqrt(pow(hm_maxR[1][0] - gt_kp[0],2)+pow(hm_maxR[0][0] - gt_kp[1],2))
#         man_dists.append(man_distR)
#         print("Right crop Manhattan distance between pred and gt {} for {}".format(man_distR[0], FLAGS.keypoints_order[c]))

#         np2_ind_dists[c].append((man_distL+man_distR)/2)

    im1ps.append([int((kpL[c][0]+kpR2L[c][0])/2), int((kpL[c][1]+kpR2L[c][1])/2)]) 
    im2ps.append([int((kpR[c][0]+kpL2R[c][0])/2), int((kpR[c][1]+kpL2R[c][1])/2)]) 
    gt1ps.append([int((gtkeypointsL[c][0]+gtR2L[c][0])/2), int((gtkeypointsL[c][1]+kpR2L[c][1])/2)]) 
    gt2ps.append([int((gtkeypointsR[c][0]+gtL2R[c][0])/2), int((gtkeypointsR[c][1]+gtL2R[c][1])/2)]) 
    man_distL = np.sqrt(pow(im1ps[c][0] - gt1ps[c][0],2) + pow(im1ps[c][1] - gt1ps[c][1],2))
    man_dists.append(man_distL)
    # print("Left crop Manhattan distance between pred and gt {} for {}".format(man_distL, FLAGS.keypoints_order[c]))
    man_distR = np.sqrt(pow(im2ps[c][0] - gt2ps[c][0],2) + pow(im2ps[c][1] - gt2ps[c][1],2))
    man_dists.append(man_distR)
    # print("Right crop Manhattan distance between pred and gt {} for {}".format(man_distR, FLAGS.keypoints_order[c]))
    np2_ind_dists[c].append((man_distL+man_distR)/2)